In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [28]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))

In [29]:
mean = train_data.data.float().mean()/255
std = train_data.data.float().std()/255

In [30]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

In [31]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)

test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

In [32]:
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=32,
    shuffle=True
)

In [33]:
class Perceptron(torch.nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = torch.nn.Linear(28 * 28, 512)
        self.fc2 = torch.nn.Linear(512, 512)
        self.fc3 = torch.nn.Linear(512, 10)
    def forward(self, input_):
        input_ = input_.view(-1, 28*28)
        h1 = F.relu(self.fc1(input_))
        h2 = F.relu(self.fc2(h1))
        h3 = self.fc3(h2)
        return F.log_softmax(h3, dim=1)
model = Perceptron()

In [34]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for i, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    y_pred = model(data)
    loss = criterion(y_pred, target)
    loss.backward()
    optimizer.step()

KeyboardInterrupt: 

In [0]:
data = test_data.data.float()
y = test_data.targets

with torch.no_grad():
    y_pred = (model(data).max(1)).indices
    display(accuracy_score(y.numpy(), y_pred))